In [2]:
import json
import argparse

import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

from postprocessing.sampling_pts import get_samplingptshandler, PyfrmSamplingPts

In [31]:
meshing_cfg_path = 'configs/mesh_gen/Omesh.json'
sampling_cfg_path = 'configs/postprocessing/annulus_64_plane.json'
pyfrm = '/storage/dataset500/cylinder/cylinder.pyfrm'

meshing_cfg = json.load(open(meshing_cfg_path, 'r'))
sampling_cfg = json.load(open(sampling_cfg_path, 'r'))

L_m = meshing_cfg['generate']['magnify']

sampling_pts = {pts:get_samplingptshandler(pyfrm = pyfrm, **sampling_cfg[pts]) for pts in sampling_cfg}

In [32]:
vertices_xy, zs = PyfrmSamplingPts._extract_physical_grp_vertices(pyfrm, 'obstacle')

vertices = []
for z in zs:
    for v in vertices_xy:
        vertices.append([v[0], v[1], z])
vertices = np.array(vertices)

ch = ConvexHull(vertices)
xc = vertices[ch.vertices]

In [33]:
#for sparse sensor visualization
fig = go.Figure()
fig.add_trace(go.Mesh3d(x=xc[:, 0]/L_m, 
                        y=xc[:, 1]/L_m, 
                        z=xc[:, 2]/L_m, 
                        color="blue", 
                        opacity=.5,
                        alphahull=0))



for pts in filter(lambda x: x != 'full_field', sampling_pts):
    coords = sampling_pts[pts]._pts.reshape(-1, 3)
    fig.add_trace(go.Scatter3d(
        name=pts,
        x=coords[:,0]/L_m,
        y=coords[:,1]/L_m,
        z=coords[:,2]/L_m,
        mode='markers',
        marker={'size':2}
    ))

fig.update_layout(scene_camera={
                      'up':{'x': 0, 'y': 1, 'z': 0}
                  },
                  scene={
                      'xaxis': {'range': [-2/L_m, 8/L_m]},
                      'yaxis': {'range': [-5/L_m, 5/L_m]},
                      'zaxis': {'range': [0/L_m, 10/L_m]}
                  })
fig.update_scenes(xaxis_title_text="x̄", yaxis_title_text="ȳ", zaxis_title_text="z̄")

In [11]:
#for plane measurement visualization
fig = go.Figure()
fig.add_trace(go.Mesh3d(x=xc[:, 0], 
                        y=xc[:, 1], 
                        z=xc[:, 2], 
                        color="blue", 
                        opacity=.5,
                        alphahull=0))


coords = sampling_pts[pts]._pts
yshape, zshape = coords.shape[1:-1]
coords_xy = coords[:,:,zshape//2].reshape(-1,3)
coords_xz = coords[:,yshape//2,:].reshape(-1,3)

fig.add_trace(go.Scatter3d(
        name='xy-sensors',
        x=coords_xy[:,0],
        y=coords_xy[:,1],
        z=coords_xy[:,2],
        mode='markers',
        marker={'size':2}
    ))

fig.add_trace(go.Scatter3d(
        name='xz-sensors',
        x=coords_xz[:,0],
        y=coords_xz[:,1],
        z=coords_xz[:,2],
        mode='markers',
        marker={'size':2}
    ))
fig.update_layout(scene_camera={
                      'up':{'x': 0, 'y': 1, 'z': 0}
                  },
                  scene={
                      'xaxis': {'range': [-2, 8]},
                      'yaxis': {'range': [-5, 5]},
                      'zaxis': {'range': [0, 10]}
                  })